<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py

--2025-03-31 15:47:33--  https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10129 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.89K  --.-KB/s    in 0.001s  

2025-03-31 15:47:33 (17.8 MB/s) - ‘helper_functions.py’ saved [10129/10129]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2025-03-31 15:47:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2025-03-31 15:47:52 (55.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing data
with the help of pandas


In [5]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# suffling train data
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
# how many samples?
len(train_df),len(test_df)

(7613, 3263)

In [9]:
# Lets visualize some random samples.
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","real disaster" if target>0 else "not real disaster")
  print(f"Text: {text}")
  print("_____")

Target:1 real disaster
Text: ÛÏ@BBCEngland: A burst water main causes major flooding at a Birmingham hospital http://t.co/q4kGftC2AM http://t.co/6w6A2L4qAeÛ OMG.
_____
Target:1 real disaster
Text: Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/rkU0IDM6aQ
_____
Target:1 real disaster
Text: The Latest: More homes razed by #NorthernCalifornia wildfire http://t.co/mONiJJth7V #ZippedNews http://t.co/0yXBB5dzw5
_____
Target:0 not real disaster
Text: when you're taking a shower and someone flushes the toilet and you have .1 second to GTFO or you get burned??????????????????????????????????????????????????
_____
Target:1 real disaster
Text: #?? #?? #??? #??? Suicide bomber kills 15 in Saudi security site mosque - Reuters  http://t.co/txg7K2DO9v
_____


In [10]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## split data in to training and validation set

In [11]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1, # use 10% of training data for validation
                                                                       random_state=42)

In [12]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text vectorizarion (Tokenization)
converting text into numbers
*   Tokenization: Its a direct mapping of token to a number
*   Embedding: Create a matrix of feature vector for each token.



In [14]:
import tensorflow as tf
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=None,
                                                  standardize='lower_and_strip_punctuation',
                                                  split='whitespace',
                                                  ngrams=None, #create groups of n-words
                                                  output_mode='int',
                                                  output_sequence_length=None, # how long do u want your sequence to be?
                                                  pad_to_max_tokens=False
                                                  )


### find the average number of tokens or words in training samples

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# setup text vectorization variables
max_vocab_length=10000 # Maximum number of words to have in our vocabolary
max_length=15 #max length our sentences will be
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode='int',
                                                  output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# lets create a sample sentences and tokenize it
sample_sentence="The recent earthquake has destroyed the entire city"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2756,  290,   41,  351,    2,  855,  182,    0,    0,    0,
           0,    0,    0,    0]])>

In [19]:
# selecting a random sample from the train dataset and tokenize it
import random
random_sentence=random.choice(train_sentences)
print(f"Random Sample: {random_sentence}")
print(f"Vectorized sample: {text_vectorizer([random_sentence])}")

Random Sample: But no lies though. It's pays to be the oldest sometimes. Like being the first to get a car and have no curfew. #freedom #donthate
Vectorized sample: [[  30   40 1125  804   37 4949    5   21    2    1 1361   25  121    2
    97]]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # Get all the unique words in the training dataset
top_5_words=words_in_vocab[:5] # get the top 5 most common words
bottom_5_words=words_in_vocab[-5:] # get the least 5 common words
print(f"Words in Vocab: {len(words_in_vocab)}")
print(f"Top 5 words: {top_5_words}")
print(f"Least 5 words: {bottom_5_words}")

Words in Vocab: 10000
Top 5 words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Least 5 words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### creating an Embedding using Embedding layer

*   input_dim=the size of our vocabolary
*   output_dim= the size of the output embedding vector
*   input_length= size of the sequences being passed to the embedding layer




In [23]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, # output shape
                           input_length=max_length # how long is each input
                           )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
# embed the random sample. i.e turn into dense vector of fixed size
print("Random sample: ",random_sentence)
print("After Embedding:...................")
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Random sample:  But no lies though. It's pays to be the oldest sometimes. Like being the first to get a car and have no curfew. #freedom #donthate
After Embedding:...................


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00837482,  0.01029534,  0.03291697, ...,  0.0437442 ,
          0.00378431,  0.00742769],
        [-0.04698978,  0.04754857,  0.00323988, ..., -0.00607328,
          0.00073556, -0.00725014],
        [-0.01731915, -0.00736452,  0.01988697, ..., -0.01195183,
          0.0252541 ,  0.02620551],
        ...,
        [ 0.03138887,  0.01023704,  0.01505382, ..., -0.01080082,
          0.01473472, -0.04303701],
        [-0.01767121, -0.00966778,  0.0292774 , ...,  0.0240004 ,
          0.03033376,  0.001537  ],
        [ 0.01871571,  0.02703046, -0.00784441, ...,  0.00883923,
         -0.01904625,  0.04671459]]], dtype=float32)>

In [32]:
# checkout single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-8.37482139e-03,  1.02953427e-02,  3.29169743e-02, -4.22259346e-02,
         4.40566614e-03, -3.52793224e-02, -3.08459532e-02, -1.21245496e-02,
         4.21761535e-02,  2.05176957e-02, -2.63425838e-02, -2.81094797e-02,
         2.38079093e-02, -1.79206245e-02, -3.31430659e-02,  4.53673340e-02,
         8.86640698e-03,  1.60827078e-02,  2.63924859e-02, -3.91377695e-02,
         3.21503766e-02, -2.03542002e-02,  3.53961028e-02, -1.50354989e-02,
         3.54953669e-02, -2.91337725e-02,  3.90578024e-02, -3.09333205e-02,
         3.93949188e-02,  4.93893884e-02, -1.14895478e-02,  3.98796238e-02,
         4.32227887e-02,  2.14367844e-02,  1.20264515e-02, -2.47557163e-02,
        -9.12836939e-03,  4.75024842e-02,  3.73099558e-02,  2.26279013e-02,
         1.17512569e-02, -1.40079260e-02,  4.76523526e-02, -4.24925573e-02,
         4.59930040e-02,  4.24302742e-03,  7.01614469e-03,  2.52596401e-02,
        -5.20944595e-04, -3.77306119e-0

This is the embedding for the first word of sentence